In [1]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import backtesting
backtesting.set_bokeh_output(notebook=False)
import warnings
warnings.filterwarnings("ignore")
warnings.warn('Jupyter Notebook detected.')
import yfinance as yf
import datetime as dt
import talib
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report


c:\Users\l.cespedes\SPP4backtesting\venv\Lib\site-packages\backtesting\_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
btc= yf.Ticker("BTC-USD")
btc_data = btc.history(start = dt.datetime(2015,1,1),
                       end=dt.datetime(2025,1,1), 
                       interval="1d")
btc_data

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2015-01-01 00:00:00+00:00,320.434998,320.434998,314.002991,314.248993,8036550,0.0,0.0
2015-01-02 00:00:00+00:00,314.079010,315.838989,313.565002,315.032013,7860650,0.0,0.0
2015-01-03 00:00:00+00:00,314.846008,315.149994,281.082001,281.082001,33054400,0.0,0.0
2015-01-04 00:00:00+00:00,281.145996,287.230011,257.612000,264.195007,55629100,0.0,0.0
2015-01-05 00:00:00+00:00,265.084015,278.341003,265.084015,274.473999,43962800,0.0,0.0
...,...,...,...,...,...,...,...
2024-12-27 00:00:00+00:00,95704.976562,97294.843750,93310.742188,94164.859375,52419934565,0.0,0.0
2024-12-28 00:00:00+00:00,94160.187500,95525.898438,94014.289062,95163.929688,24107436185,0.0,0.0
2024-12-29 00:00:00+00:00,95174.054688,95174.875000,92881.789062,93530.226562,29635885267,0.0,0.0


In [7]:
talib.ROC(btc_data.Close)

Date
2015-01-01 00:00:00+00:00          NaN
2015-01-02 00:00:00+00:00          NaN
2015-01-03 00:00:00+00:00          NaN
2015-01-04 00:00:00+00:00          NaN
2015-01-05 00:00:00+00:00          NaN
                               ...    
2024-12-27 00:00:00+00:00   -11.282904
2024-12-28 00:00:00+00:00    -4.875584
2024-12-29 00:00:00+00:00    -4.062661
2024-12-30 00:00:00+00:00    -5.230086
2024-12-31 00:00:00+00:00    -3.903866
Length: 3653, dtype: float64

In [27]:

data = btc_data.iloc[:,:5]
data['Target'] = np.where(data['Close'].shift(-1) > data['Close'], 1, 0)
data.dropna(inplace=True)
data['AD'] = talib.AD(data.High, data.Low, data.Close, data.Volume)
data

,Open,High,Low,Close,Volume,Target,AD
Date,,,,,,,
2015-01-01 00:00:00+00:00,320.434998,320.434998,314.002991,314.248993,8036550,1,-7.421809e+06
2015-01-02 00:00:00+00:00,314.079010,315.838989,313.565002,315.032013,7860650,0,-5.140223e+06
2015-01-03 00:00:00+00:00,314.846008,315.149994,281.082001,281.082001,33054400,0,-3.819462e+07
2015-01-04 00:00:00+00:00,281.145996,287.230011,257.612000,264.195007,55629100,1,-6.909507e+07
2015-01-05 00:00:00+00:00,265.084015,278.341003,265.084015,274.473999,43962800,1,-5.077977e+07
...,...,...,...,...,...,...,...
2024-12-27 00:00:00+00:00,95704.976562,97294.843750,93310.742188,94164.859375,52419934565,1,5.541439e+12
2024-12-28 00:00:00+00:00,94160.187500,95525.898438,94014.289062,95163.929688,24107436185,0,5.554001e+12
2024-12-29 00:00:00+00:00,95174.054688,95174.875000,92881.789062,93530.226562,29635885267,0,5.541126e+12


In [28]:
X = data[['AD']]
y = data['Target']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [34]:
model = KNeighborsClassifier(n_neighbors=9)
model.fit(X_train, y_train)


,n_neighbors,9
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


In [35]:
y_pred = model.predict(X_test)
print("Precisión:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Precisión: 0.5471956224350205
              precision    recall  f1-score   support

           0       0.50      0.52      0.51       330
           1       0.59      0.57      0.58       401

    accuracy                           0.55       731
   macro avg       0.54      0.55      0.54       731
weighted avg       0.55      0.55      0.55       731

